In [209]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [210]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [211]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [212]:
train['Expenses'] = train.RoomService + train.FoodCourt + train.ShoppingMall + train.Spa + train.VRDeck
train.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'PassengerId'], axis='columns', inplace=True)
train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,Expenses
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,False,0.0
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,True,736.0
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,False,10383.0
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,False,5176.0
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,True,1091.0


In [213]:
test['Expenses'] = test.RoomService + test.FoodCourt + test.ShoppingMall + test.Spa + test.VRDeck
test.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name', 'PassengerId'], axis='columns', inplace=True)
test.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Expenses
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,2832.0
2,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,7418.0
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,645.0


In [214]:
train = pd.get_dummies(data=train, columns=['HomePlanet', 'Destination'])
train.head()

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,B/0/P,39.0,False,False,0.0,0,1,0,0,0,1
1,False,F/0/S,24.0,False,True,736.0,1,0,0,0,0,1
2,False,A/0/S,58.0,True,False,10383.0,0,1,0,0,0,1
3,False,A/0/S,33.0,False,False,5176.0,0,1,0,0,0,1
4,False,F/1/S,16.0,False,True,1091.0,1,0,0,0,0,1


In [215]:
test = pd.get_dummies(data=test, columns=['HomePlanet', 'Destination'])
test.head()

,CryoSleep,Cabin,Age,VIP,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,G/3/S,27.0,False,0.0,1,0,0,0,0,1
1,False,F/4/S,19.0,False,2832.0,1,0,0,0,0,1
2,True,C/0/S,31.0,False,0.0,0,1,0,1,0,0
3,False,C/1/S,38.0,False,7418.0,0,1,0,0,0,1
4,False,F/5/S,20.0,False,645.0,1,0,0,0,0,1


In [216]:
for c in ['CryoSleep', 'VIP', 'Transported']:
    train[c].replace({True: 1, False: 0}, inplace=True)
    
train.sample(15)

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
5921,0.0,G/1015/S,38.0,0.0,1,773.0,1,0,0,0,0,1
6143,0.0,F/1237/S,26.0,0.0,0,1686.0,0,0,1,1,0,0
5720,0.0,F/1254/P,33.0,0.0,0,1815.0,0,0,1,0,0,1
3683,0.0,F/817/P,28.0,0.0,0,673.0,1,0,0,0,0,1
7270,0.0,C/253/P,28.0,0.0,0,6527.0,0,1,0,1,0,0
2620,0.0,F/582/P,19.0,0.0,0,634.0,1,0,0,0,0,1
2469,0.0,F/547/P,24.0,0.0,0,704.0,1,0,0,0,0,1
7290,0.0,F/1617/P,18.0,0.0,0,2141.0,0,0,0,0,0,1
8237,0.0,F/1706/S,27.0,0.0,0,2050.0,1,0,0,0,0,1
99,0.0,D/2/S,21.0,0.0,0,1765.0,0,0,1,0,0,1


In [217]:
for c in ['CryoSleep', 'VIP']:
    test[c].replace({True: 1, False: 0}, inplace=True)
    
train.sample(15)

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
8100,0.0,F/1779/P,35.0,0.0,0,861.0,1,0,0,0,0,1
8232,0.0,F/1704/S,37.0,0.0,0,724.0,1,0,0,0,0,1
3799,0.0,G/669/S,36.0,0.0,0,882.0,1,0,0,0,0,1
3198,0.0,B/131/S,33.0,0.0,1,27086.0,0,1,0,1,0,0
259,0.0,F/65/P,57.0,0.0,0,1488.0,1,0,0,1,0,0
3203,1.0,G/555/S,44.0,0.0,1,0.0,1,0,0,0,1,0
6668,0.0,F/1458/P,20.0,0.0,1,2450.0,1,0,0,0,0,1
3155,1.0,G/546/S,38.0,0.0,0,0.0,1,0,0,0,0,1
1077,1.0,C/42/P,29.0,0.0,1,NaN,0,1,0,0,0,1
5769,1.0,A/58/P,34.0,0.0,1,0.0,0,1,0,1,0,0


In [218]:
train.dropna(axis='rows', inplace=True)
train.sample(10)

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
6517,1.0,F/1430/P,32.0,0.0,1,0.0,0,0,1,1,0,0
399,0.0,F/78/S,40.0,0.0,0,873.0,1,0,0,0,1,0
4218,1.0,F/846/S,28.0,0.0,1,0.0,0,0,1,0,0,1
2141,0.0,F/448/S,35.0,0.0,0,617.0,0,0,1,0,0,1
24,0.0,E/0/S,10.0,0.0,1,0.0,1,0,0,0,0,1
5511,1.0,G/953/P,11.0,0.0,1,0.0,1,0,0,0,0,1
4380,0.0,F/876/S,24.0,0.0,0,703.0,1,0,0,0,0,1
3389,1.0,D/124/P,16.0,0.0,1,0.0,0,0,1,0,0,1
6092,1.0,G/1044/S,49.0,0.0,1,0.0,1,0,0,0,1,0
5314,0.0,E/359/P,18.0,0.0,1,1609.0,1,0,0,0,0,1


In [219]:
test.dropna(axis='rows', inplace=True)

In [220]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaling_columns = ['Age', 'Expenses']

In [221]:
train[scaling_columns] = scaler.fit_transform(train[scaling_columns])
train.head()

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0.0,B/0/P,0.493671,0.0,0,0.000000,0,1,0,0,0,1
1,0.0,F/0/S,0.303797,0.0,1,0.020452,1,0,0,0,0,1
2,0.0,A/0/S,0.734177,1.0,0,0.288521,0,1,0,0,0,1
3,0.0,A/0/S,0.417722,0.0,0,0.143830,0,1,0,0,0,1
4,0.0,F/1/S,0.202532,0.0,1,0.030317,1,0,0,0,0,1


In [222]:
test[scaling_columns] = scaler.fit_transform(test[scaling_columns])

In [223]:
cabin = 'E/556/S'
cabin = ''.join([i for i in cabin if not i.isdigit()])
cabin = cabin.replace('/', '')
cabin

'ES'

In [224]:
def clean_cabin(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.replace('/', '')

In [225]:
train['Cabin'] = train['Cabin'].apply(clean_cabin)
train.sample(5)

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
1283,0.0,ES,0.316456,0.0,1,0.027704,1,0,0,0,0,1
6203,0.0,FS,0.518987,0.0,1,0.020341,1,0,0,0,0,1
5849,1.0,AP,0.569620,0.0,1,0.000000,0,1,0,0,0,1
3589,1.0,ES,0.392405,0.0,0,0.000000,0,0,1,0,0,1
1347,0.0,BP,0.367089,0.0,0,0.192125,0,1,0,1,0,0


In [226]:
test['Cabin'] = test['Cabin'].apply(clean_cabin)
train.sample(5)

,CryoSleep,Cabin,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
7014,0.0,BS,0.417722,0.0,0,0.069219,0,1,0,0,0,1
124,0.0,DS,0.455696,0.0,0,0.076083,0,0,1,0,0,1
896,0.0,FS,0.240506,0.0,0,0.049157,0,0,1,0,0,1
5474,1.0,CS,0.341772,0.0,1,0.000000,0,1,0,0,0,1
6277,1.0,GS,0.215190,0.0,1,0.000000,1,0,0,1,0,0


In [227]:
train = pd.get_dummies(data=train, columns=['Cabin'])
train.sample(5)

,CryoSleep,Age,VIP,Transported,Expenses,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,...,Cabin_CS,Cabin_DP,Cabin_DS,Cabin_EP,Cabin_ES,Cabin_FP,Cabin_FS,Cabin_GP,Cabin_GS,Cabin_TP
2555,0.0,0.240506,0.0,1,0.014033,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3065,0.0,0.177215,0.0,0,0.018673,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1302,0.0,0.215190,0.0,0,0.027121,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1304,0.0,0.278481,0.0,1,0.022508,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5185,0.0,0.544304,0.0,0,0.039848,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0


In [228]:
test = pd.get_dummies(data=test, columns=['Cabin'])

In [229]:
X_train = train.drop('Transported', axis='columns')
y_train = train['Transported']

In [230]:
X_train.shape, test.shape

((7084, 25), (3491, 26))

In [231]:
for i in range(25):
    print(f'{X_train.columns[i]}: {test.columns[i]}')

print(f'None: {test.columns[25]}')

CryoSleep: CryoSleep
Age: Age
VIP: VIP
Expenses: Expenses
HomePlanet_Earth: HomePlanet_Earth
HomePlanet_Europa: HomePlanet_Europa
HomePlanet_Mars: HomePlanet_Mars
Destination_55 Cancri e: Destination_55 Cancri e
Destination_PSO J318.5-22: Destination_PSO J318.5-22
Destination_TRAPPIST-1e: Destination_TRAPPIST-1e
Cabin_AP: Cabin_AP
Cabin_AS: Cabin_AS
Cabin_BP: Cabin_BP
Cabin_BS: Cabin_BS
Cabin_CP: Cabin_CP
Cabin_CS: Cabin_CS
Cabin_DP: Cabin_DP
Cabin_DS: Cabin_DS
Cabin_EP: Cabin_EP
Cabin_ES: Cabin_ES
Cabin_FP: Cabin_FP
Cabin_FS: Cabin_FS
Cabin_GP: Cabin_GP
Cabin_GS: Cabin_GS
Cabin_TP: Cabin_TP
None: Cabin_TS


In [232]:
X_train['Cabin_TS'] = np.zeros(7084)

In [233]:
model = keras.Sequential([
    keras.layers.Dense(20, input_shape=(26,), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20)

Epoch 1/20
222/222 [==============================] - 1s 2ms/step - loss: 0.6185 - accuracy: 0.6776
Epoch 2/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5503 - accuracy: 0.7212
Epoch 3/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5330 - accuracy: 0.7261
Epoch 4/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5257 - accuracy: 0.7297
Epoch 5/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.7318
Epoch 6/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5176 - accuracy: 0.7345
Epoch 7/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5155 - accuracy: 0.7396
Epoch 8/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5134 - accuracy: 0.7396
Epoch 9/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5124 - accuracy: 0.7407
Epoch 10/20
222/222 [==============================] - 0s 2ms/step - loss: 0.5109 - accuracy: 0.7428

In [234]:
y_predict = model.predict(test)
y_predict

array([[0.7090809 ],
       [0.32164782],
       [0.9962896 ],
       ...,
       [0.30694562],
       [0.6952569 ],
       [0.6866892 ]], dtype=float32)

In [235]:
y_predict_filtered = []

for p in y_predict:
    if p[0] > .5:
        y_predict_filtered.append(1)
    else:
        y_predict_filtered.append(0)
        
y_predict_filtered = np.asarray(y_predict_filtered)
y_predict_filtered.shape

(3491,)